In [49]:
!pip install pulp

# Integer Programming

Integer programming is a mathematical optimization technique used when some or all of the variables in a linear programming problem must take integer values. When all variables must be either 0 or 1 (binary), we call it Binary Integer Programming.

## Common applications of Integer Programming:

1. **Facility location problems** - deciding where to build facilities
2. **Resource allocation** - assigning limited resources to tasks
3. **Scheduling** - allocating time slots to activities
4. **Transportation and logistics** - optimizing routes and shipments
5. **Manufacturing** - production planning and inventory control
6. **Financial planning** - portfolio optimization with discrete investments

## Example 1: Facility Location Problem

Our company already has factories in two cities (City A and City B) and is considering expanding them. We also want to build a warehouse (only one).

Decision variables:
- Build factory in City A? (x1): Added value: $9 million, capital required: $6 million
- Build factory in City B? (x2): Added value: $5 million, capital required: $3 million
- Build warehouse in City A? (x3): Added value: $6 million, capital required: $5 million
- Build warehouse in City B? (x4): Added value: $4 million, capital required: $2 million

Available capital: $10 million

Additional constraint: A warehouse can only be built in a city if there is also a factory there.




![image](https://github.com/AdoHaha/automation_robotics_lab/blob/master/lab3a_expl.png?raw=1)


In [50]:
    # Let's visualize the problem:
    #
    #                 City A                  City B
    #                 ------                 ------
    # Factory:     $9M value              $5M value
    #              $6M cost               $3M cost
    #                  |                      |
    #                  v                      v
    # Warehouse:   $6M value              $4M value
    #              $5M cost               $2M cost
    #
    # Total capital available: $10M
    # Must choose max 1 warehouse, and only where factory exists

In [51]:
import pulp
from pulp import *

# Create the model
prob = LpProblem("Facility_Location_Problem", LpMaximize)

In [52]:
# Decision variables (binary: 0 or 1)
x1 = LpVariable("Factory_CityA", cat=LpBinary)
x2 = LpVariable("Factory_CityB", cat=LpBinary)
x3 = LpVariable("Warehouse_CityA", cat=LpBinary)
x4 = LpVariable("Warehouse_CityB", cat=LpBinary)

In [53]:
# Objective function: maximize total value
prob += 9*x1 + 5*x2 + 6*x3 + 4*x4, "Total_Added_Value"

In [54]:
# Constraints
# 1. Capital constraint
prob += 6*x1 + 3*x2 + 5*x3 + 2*x4 <= 10, "Available_Capital"

# 2. Only one warehouse
prob += x3 + x4 <= 1, "Maximum_One_Warehouse"

# 3. Warehouse can only be built if factory exists in same city
prob += x1 - x3 >= 0, "CityA_Warehouse_Requires_Factory"
prob += x2 - x4 >= 0, "CityB_Warehouse_Requires_Factory"

In [55]:
# Helper function to print the solution
def print_solution(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    for v in p.variables():
        print(v.name, "=", v.varValue)
    print("Objective value =", value(p.objective))

In [56]:
# Solve the problem
print_solution(prob)

Status: Optimal
Factory_CityA = 1.0
Factory_CityB = 1.0
Warehouse_CityA = 0.0
Warehouse_CityB = 0.0
Objective value = 14.0


## Binary Problems

Binary integer programming is used for "yes/no" decision problems such as:
- Which route to choose
- Which truck to use
- Whether to make an investment

We can also model more complex logical conditions using binary variables.

## Example 2: Modeling Logical Constraints

Sometimes we need to model that ONE of two constraints must be satisfied, but not necessarily both.

For example, maximize x + y where 0 ≤ x ≤ 10, 0 ≤ y ≤ 10, and EITHER (x + y ≤ 3) OR (3y + x ≤ 3).

We can use a binary variable z to implement this logical OR:

In [57]:
prob2 = LpProblem("Alternative_Constraints_Problem", LpMaximize)

# Continuous variables
x = LpVariable("x", 0, 10, cat=LpContinuous)
y = LpVariable("y", 0, 10, cat=LpContinuous)

# Binary variable to implement logical OR
z = LpVariable("ignore_first_constraint", cat=LpBinary)

# Objective function
prob2 += x + y, "Simple_Sum"

# If z=1, first constraint is relaxed (ignored)
# If z=0, second constraint is relaxed (ignored)
M = 10000  # A very large number
prob2 += x + y <= 3 + M*z, "first_constraint"
prob2 += 3*x - y <= 3 + M*(1-z), "second_constraint"

# Solve and display results
print_solution(prob2)
print(prob2)

Status: Optimal
ignore_first_constraint = 1.0
x = 4.3333333
y = 10.0
Objective value = 14.3333333
Alternative_Constraints_Problem:
MAXIMIZE
1*x + 1*y + 0
SUBJECT TO
first_constraint: - 10000 ignore_first_constraint + x + y <= 3

second_constraint: 10000 ignore_first_constraint + 3 x - y <= 10003

VARIABLES
0 <= ignore_first_constraint <= 1 Integer
x <= 10 Continuous
y <= 10 Continuous



## Example 3: Scheduling Problem

Let's consider a scheduling problem where we need to assign employees to workdays.

### Problem description:
- We have three employees: Anna, Kate, and Peter
- Each employee can work up to 3 days per week
- Daily rates are: Anna ($150), Kate ($160), Peter ($140)
- Anna can't work on Monday, Peter can't work on Thursday and Friday
- On Tuesday we need 2 employees, on other days we need 1 employee
- Goal: minimize the total cost

This is a perfect application for integer programming with binary variables.

In [58]:
prob3 = LpProblem("Scheduling_Problem", LpMinimize)

# Define data
days = ["mon", "tue", "wed", "thu", "fri"]
employees = ["Anna", "Kate", "Peter"]
costs = [150, 160, 140]  # daily rates

# Create binary decision variables for each employee on each day
schedule = LpVariable.dicts("Schedule", (employees, days), cat="Binary")

In [59]:
# Objective function: minimize total cost
prob3 += lpSum([costs[i] * lpSum([schedule[employee][day] for day in days])
                for i, employee in enumerate(employees)])

In [60]:
# Constraint: required number of employees each day
required_employees = [1, 2, 1, 1, 1]  # mon, tue, wed, thu, fri
for day, required in zip(days, required_employees):
    prob3 += lpSum([schedule[employee][day] for employee in employees]) == required

# Constraint: employee availability
prob3 += schedule["Anna"]["mon"] == 0  # Anna can't work Monday
prob3 += schedule["Peter"]["thu"] == 0  # Peter can't work Thursday
prob3 += schedule["Peter"]["fri"] == 0  # Peter can't work Friday

# Constraint: maximum workdays per employee
for employee in employees:
    prob3 += lpSum([schedule[employee][day] for day in days]) <= 3

In [61]:
# Solve and print the solution
print_solution(prob3)

Status: Optimal
Schedule_Anna_fri = 1.0
Schedule_Anna_mon = 0.0
Schedule_Anna_thu = 1.0
Schedule_Anna_tue = 1.0
Schedule_Anna_wed = 0.0
Schedule_Kate_fri = 0.0
Schedule_Kate_mon = 0.0
Schedule_Kate_thu = 0.0
Schedule_Kate_tue = 0.0
Schedule_Kate_wed = 0.0
Schedule_Peter_fri = 0.0
Schedule_Peter_mon = 1.0
Schedule_Peter_thu = 0.0
Schedule_Peter_tue = 1.0
Schedule_Peter_wed = 1.0
Objective value = 870.0


# Excercise 1

1) Extend the example to four Emploeeys: there is also Mark, who can work on any day and his rate is the lowest: $100 per day but he can work up to 4 days per week.

2) Add a constraint(s) that Mark can work only if Anna is not there (use logical constraint)

3) Bonus:  Make the program interactive: add a checkbox to show which employees are available (and which are not)


In [62]:
prob3 = LpProblem("Scheduling_Problem", LpMinimize)  # Tworzymy problem minimalizacji kosztu

# --- Dane wejściowe ---
days = ["mon", "tue", "wed", "thu", "fri"]          # dni tygodnia
employees = ["Anna", "Kate", "Peter", "Mark"]       # pracownicy
costs = [150, 160, 140, 100]                        # dzienne stawki
mdays = [3, 3, 3, 4]                                # maks. dni pracy tygodniowo

# --- Zmienne decyzyjne ---
# Binary: 1 jeśli pracownik pracuje danego dnia
schedule = LpVariable.dicts("Schedule", (employees, days), cat="Binary")

# --- Funkcja celu ---
# Minimalizacja całkowitego kosztu pracy
prob3 += lpSum([costs[i] * lpSum([schedule[employee][day] for day in days])
                for i, employee in enumerate(employees)])

# --- Ograniczenia: wymagani liczba pracowników każdego dnia ---
required_employees = [1, 2, 1, 1, 1]  # mon-fri
for day, required in zip(days, required_employees):
    prob3 += lpSum([schedule[employee][day] for employee in employees]) == required

# --- Ograniczenia dostępności konkretnych pracowników ---
prob3 += schedule["Anna"]["mon"] == 0  # Anna nie może pracować w poniedziałek
prob3 += schedule["Peter"]["thu"] == 0  # Peter nie może pracować w czwartek
prob3 += schedule["Peter"]["fri"] == 0  # Peter nie może pracować w piątek

# --- Ograniczenia: Anna vs Mark ---
# Tworzymy dodatkowe zmienne binarne, aby zaznaczyć dni, w których Anna nie pracuje
anna_not_working_today = LpVariable.dicts("Anna_Not_Working", days, cat="Binary")
M = 10000  # duża liczba dla techniki big-M

for day in days:
  # Anna i zmienna "Anna_Not_Working" nie mogą być jednocześnie 1
  prob3 += schedule["Anna"][day] + anna_not_working_today[day] <= 1, f"Anna_or_NotWorking_{day}"
  # Mark może pracować tylko jeśli Anna nie pracuje (big-M)
  prob3 += schedule["Mark"][day] <= M * anna_not_working_today[day], f"Mark_Needs_Anna_Off_{day}"

# --- Maksymalna liczba dni pracy dla każdego pracownika ---
for employee, md in zip(employees, mdays):
   prob3 += lpSum([schedule[employee][day] for day in days]) <= md

# --- Rozwiązanie ---
print_solution(prob3)  # drukuje harmonogram (uwaga: funkcja musi być zdefiniowana)


Status: Optimal
Anna_Not_Working_fri = 1.0
Anna_Not_Working_mon = 1.0
Anna_Not_Working_thu = 1.0
Anna_Not_Working_tue = 1.0
Anna_Not_Working_wed = 1.0
Schedule_Anna_fri = 0.0
Schedule_Anna_mon = 0.0
Schedule_Anna_thu = 0.0
Schedule_Anna_tue = 0.0
Schedule_Anna_wed = 0.0
Schedule_Kate_fri = 0.0
Schedule_Kate_mon = 0.0
Schedule_Kate_thu = 0.0
Schedule_Kate_tue = 0.0
Schedule_Kate_wed = 0.0
Schedule_Mark_fri = 1.0
Schedule_Mark_mon = 0.0
Schedule_Mark_thu = 1.0
Schedule_Mark_tue = 1.0
Schedule_Mark_wed = 1.0
Schedule_Peter_fri = 0.0
Schedule_Peter_mon = 1.0
Schedule_Peter_thu = 0.0
Schedule_Peter_tue = 1.0
Schedule_Peter_wed = 0.0
Objective value = 680.0


## Extension: Hourly Scheduling

We can extend the scheduling problem to assign specific hours rather than just days.

### Revised problem:
- Each employee can work up to 24 hours per week
- Hourly rates: Anna ($15), Kate ($16), Peter ($14)
- Tuesday requires 12 hours of work, other days require 8 hours
- Other constraints remain the same

This requires integer (not just binary) variables to represent hours worked.

In [63]:
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, lpSum

# Dane
employees = ["Anna", "Peter", "John"]
days = ["mon", "tue", "wed", "thu", "fri"]
hourly_rates = [15, 16, 14]  # odpowiada kolejno Anna, Peter, John
hours_required = [8, 12, 8, 8, 8]  # mon, tue, wed, thu, fri

# Problem
prob4 = LpProblem("Hourly_Scheduling_Problem", LpMinimize)

# Zmienne
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=24, cat=LpInteger)

# Funkcja celu: minimalizacja kosztów
prob4 += lpSum([hourly_rates[i] * lpSum([hours[employee][day] for day in days])
                for i, employee in enumerate(employees)])

# Wymagane godziny każdego dnia
for day, required in zip(days, hours_required):
    prob4 += lpSum([hours[employee][day] for employee in employees]) == required

# Ograniczenia dostępności
prob4 += hours["Anna"]["mon"] == 0
prob4 += hours["Peter"]["thu"] == 0
prob4 += hours["Peter"]["fri"] == 0

# Maksymalna liczba godzin na tydzień
for employee in employees:
    prob4 += lpSum([hours[employee][day] for day in days]) <= 24

# Rozwiązanie
prob4.solve()

# Wydruk rozwiązania
for e in employees:
    for d in days:
        print(f"{e} on {d}: {hours[e][d].value()} hours")


Anna on mon: 0.0 hours
Anna on tue: 12.0 hours
Anna on wed: 8.0 hours
Anna on thu: 0.0 hours
Anna on fri: 0.0 hours
Peter on mon: 0.0 hours
Peter on tue: 0.0 hours
Peter on wed: 0.0 hours
Peter on thu: 0.0 hours
Peter on fri: 0.0 hours
John on mon: 8.0 hours
John on tue: 0.0 hours
John on wed: 0.0 hours
John on thu: 8.0 hours
John on fri: 8.0 hours


In [64]:
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, lpSum, LpStatus

# Dane
employees = ["Anna", "Peter", "John", "Mark"]
days = ["mon", "tue", "wed", "thu", "fri"]
hourly_rates = [15, 16, 14, 100]  # stawki $/h
hours_required = [8, 12, 8, 8, 8]  # mon-fri

# Problem
prob = LpProblem("Hourly_Scheduling_with_Mark", LpMinimize)

# Zmienne
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=24, cat=LpInteger)

# Funkcja celu: minimalizacja kosztów
prob += lpSum([hourly_rates[i] * lpSum([hours[employee][day] for day in days])
               for i, employee in enumerate(employees)])

# Wymagane godziny każdego dnia
for day, required in zip(days, hours_required):
    prob += lpSum([hours[employee][day] for employee in employees]) == required

# Ograniczenia dostępności
prob += hours["Anna"]["mon"] == 0
prob += hours["Peter"]["thu"] == 0
prob += hours["Peter"]["fri"] == 0

# Maksymalna liczba godzin na tydzień dla pozostałych pracowników
for employee in ["Anna", "Peter", "John"]:
    prob += lpSum([hours[employee][day] for day in days]) <= 24

# Ograniczenia dla Marka
prob += lpSum([hours["Mark"][day] for day in days]) <= 5  # max 5h tygodniowo

# Możemy też ograniczyć do maks 4 dni pracy:
# W tym celu wprowadzimy zmienne binarne: 1 jeśli pracuje dany dzień
from pulp import LpBinary

mark_work_day = LpVariable.dicts("MarkWorkDay", days, cat=LpBinary)
for day in days:
    prob += hours["Mark"][day] <= 24 * mark_work_day[day]  # jeśli pracuje, max 24h (nasze ograniczenie h/m day)
prob += lpSum([mark_work_day[day] for day in days]) <= 4  # max 4 dni pracy

# Rozwiązanie
prob.solve()

# Wydruk rozwiązania
print(f"Status: {LpStatus[prob.status]}\n")
for e in employees:
    for d in days:
        print(f"{e} on {d}: {hours[e][d].value()} hours")


Status: Optimal

Anna on mon: 0.0 hours
Anna on tue: 4.0 hours
Anna on wed: 8.0 hours
Anna on thu: 8.0 hours
Anna on fri: 0.0 hours
Peter on mon: 0.0 hours
Peter on tue: 0.0 hours
Peter on wed: 0.0 hours
Peter on thu: 0.0 hours
Peter on fri: 0.0 hours
John on mon: 8.0 hours
John on tue: 8.0 hours
John on wed: 0.0 hours
John on thu: 0.0 hours
John on fri: 8.0 hours
Mark on mon: 0.0 hours
Mark on tue: 0.0 hours
Mark on wed: 0.0 hours
Mark on thu: 0.0 hours
Mark on fri: 0.0 hours


## Conclusion

Integer programming is a powerful tool for solving optimization problems with discrete decisions. It's particularly useful in scheduling, where we often need to assign resources (like employees) to specific time slots subject to various constraints.

The key benefits include:
1. Ability to model logical conditions (AND, OR, IF-THEN)
2. Natural representation of indivisible resources
3. Optimal solutions for complex constraint satisfaction problems

However, integer programming problems can be computationally intensive as the number of variables increases.